In [1]:
from configparser import ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from get_timbre import get_timbre
from evaluation import evaluate_data


parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'
playlists = sp.user_playlists(user_id)
playlists = playlists['items']





playlist_id = '4iSFh1pFoe4fiuQn8vX0SY'

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:

# results = sp.playlist_tracks(playlist_id)
# tracks = results['items']
# track_ids = []
# for track in tracks:
#     track_ids.append(track['track']['id'])
# track_ids

# columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
# headings = ['danceability',
# 'energy',
# 'key',
# 'loudness',
# 'mode',
# 'speechiness',
# 'acousticness',
# 'instrumentalness',
# 'liveness',
# 'valence',
# 'tempo',
# 'id',
# 'duration_ms',
# 'time_signature',
# 'track_title',
# 'album_title',
# 'album_artist',
# 'track_number',
# 'total_tracks']

# playlist_data = pd.DataFrame(columns=headings)

# playlist_length = len(track_ids)

# i = 0
# for track in track_ids:
#     results = sp.audio_features(track)
#     if results[0]: 
#         features = results[0]
    
#     features_matrix = pd.DataFrame.from_records(features, index=[0])
    
#     # Remove unneeded columns
#     features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
#     # print(features_matrix)

#     # Add track title
#     track_title = tracks[i]['track']['name']
#     features_matrix['track_title'] = track_title

#     # Add album title
#     album_title = tracks[i]['track']['album']['name']
#     features_matrix['album_title'] = album_title

#     # Add artist name
#     name = tracks[i]['track']['album']['artists'][0]['name']
#     features_matrix['album_artist'] = name

#     # Add track number and total tracks
#     features_matrix['track_number'] = tracks[i]['track']['track_number']
#     features_matrix['total_tracks'] = playlist_length

#     # Add album as a row to the data df
#     # print(features_matrix,'\n') 

#     playlist_data = pd.concat([playlist_data, features_matrix])
#     i += 1

# playlist_data = playlist_data.reset_index(drop=True)
# # playlist_data.to_csv('./data/playlist_data.csv')
# playlist_data_full = pd.DataFrame(playlist_data)

# timbre = pd.DataFrame(columns = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end'])

# i = 1
# for track in track_ids:
#     print('(' + str(i) + '/'+ str(playlist_length) + ') - ' + 'Getting timbre for: '+ track + '.')
#     track_timbre = get_timbre(track, playlist_data_full)
#     timbre = pd.concat([timbre, track_timbre], axis=0)
#     i+=1

# # playlist_data = pd.concat([playlist_data, timbre], axis=0)
# timbre = timbre.reset_index(drop=True)

# playlist_data = pd.concat([playlist_data, timbre], axis=1)



# # Remove unnecessary testing columns
# columns_to_remove = ['id',
#                     'track_title',
#                     'album_title',
#                     'album_artist',
#                     'track_number',
#                     'total_tracks',
#                     'key',
#                     'mode',
#                     'duration_ms',
#                     'time_signature']
# playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

In [2]:
playlist_data_full.to_csv('./playlist_data/'+ playlist_id +'.csv', index=False)

In [2]:
import pandas as pd

playlist_data_full = pd.read_csv('./playlist_data/'+ playlist_id +'.csv')
# playlist_data_full

In [3]:
from evaluation import evaluate_data
# eval_df = pd.DataFrame()
eval_df = evaluate_data(playlist_data_full)
eval_df

c:\Users\crisp\Desktop\Uni\AI\evaluation.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songA['key'] = (songA['key'] + 3) % 12
c:\Users\crisp\Desktop\Uni\AI\evaluation.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songB['key'] = (songB['key'] + 3) % 12


,keyTransition,tempoTransition,trackNumTransition,danceabilityTransition,energyTransition,speechinessTransition,acousticnessTransition,instrumentalnessTransition,livenessTransition,valenceTransition
0,0.275862,0.356322,0.126437,0.287356,0.275862,0.103448,0.034483,0.275862,0.08046,0.37931


In [14]:
## Load trained model
model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
playlist_data_full['order'] = model.predict(playlist_data.values)
playlist_data_full = playlist_data_full.sort_values(by=['order'])
playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
playlist_data_full

C:\Users\crisp\anaconda3\envs\AI_general\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\crisp\anaconda3\envs\AI_general\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator ExtraTreesRegressor from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
56,0.865,0.521,0,-6.932,1,0.0371,0.54800,0.000115,0.0989,0.748,129.059,1qDrWA6lyx8cLECdZE7TV7,244885,4,Somebody That I Used To Know,Making Mirrors,Gotye,3,87,1
10,0.620,0.936,7,-4.593,1,0.0308,0.02610,0.000000,0.0679,0.763,97.527,4YwbSZaYeYja8Umyt222Qf,175747,4,You Can't Hurry Love - 2016 Remaster,"Hello, I Must Be Going! (Deluxe Edition)",Phil Collins,5,87,2
34,0.851,0.551,4,-5.177,1,0.0396,0.15100,0.000000,0.0827,0.807,131.922,66S14BkJDxgkYxLl5DCqOz,219467,4,I'm Gonna Be (500 Miles),Finest,The Proclaimers,1,87,3
8,0.828,0.676,7,-6.132,1,0.0320,0.03580,0.000000,0.0513,0.901,117.041,2mc3KhEpFAh8saVFVNb1hC,235627,4,Don't Stop Movin',Sunshine,S Club 7,1,87,4
67,0.682,0.641,8,-8.915,1,0.0386,0.56800,0.000000,0.2240,0.826,141.002,09OojFvtrM9YRzRjnXqJjA,126146,4,Rockin' Around the Christmas Tree,A Brenda Lee Christmas,Brenda Lee,1,87,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,0.578,0.866,1,-3.804,1,0.0619,0.00701,0.000000,0.2570,0.619,128.038,4kbj5MwxO1bq9wjT5g9HaA,199080,4,Shut Up and Dance,TALKING IS HARD,WALK THE MOON,3,87,83
22,0.689,0.912,9,-3.673,1,0.0302,0.17400,0.000243,0.2980,0.982,125.940,06rPFgQsKS607RhYIqCmGq,216347,4,September,"Now, Then & Forever","Earth, Wind & Fire",3,87,84
38,0.675,0.895,4,-4.904,1,0.0484,0.31900,0.000000,0.3690,0.824,108.093,1fn6EFBBNiDVhL3DxDDJTD,213600,4,Bring It All Back,S Club,S Club 7,1,87,85
75,0.351,0.928,1,-3.710,1,0.0759,0.00113,0.000000,0.0988,0.239,148.030,0eGsygTp906u18L0Oimnem,222076,4,Mr. Brightside,Hot Fuss,The Killers,2,87,86


In [16]:
eval_df['sorted'] = evaluate_data(playlist_data_full)
eval_df['difference'] = (eval_df['sorted'] - eval_df['original'])
eval_df

c:\Users\crisp\Desktop\Uni\AI\evaluation.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songA['key'] = (songA['key'] + 3) % 12
c:\Users\crisp\Desktop\Uni\AI\evaluation.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  songB['key'] = (songB['key'] + 3) % 12


,original,sorted,difference
0,0.275862,0.229885,-0.045977
1,0.356322,0.252874,-0.103448
2,0.126437,0.137931,0.011494
3,0.287356,0.275862,-0.011494
4,0.275862,0.298851,0.022989
5,0.103448,0.126437,0.022989
6,0.034483,0.045977,0.011494
7,0.275862,0.298851,0.022989
8,0.080460,0.091954,0.011494
9,0.379310,0.264368,-0.114943


0.4

In [45]:
from sklearn.metrics import mean_squared_error
y_true = playlist_data_full['track_number']
y_pred = playlist_data_full['order']
mean_squared_error(y_true, y_pred)

27.1

Rearrange

In [7]:
# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))


# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST

HTTP Error for PUT to https://api.spotify.com/v1/playlists/3YwPUeHZqBrFzLzP5BF8ND/tracks with Params: {} returned 403 due to User not registered in the Developer Dashboard


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/playlists/3YwPUeHZqBrFzLzP5BF8ND/tracks:
 User not registered in the Developer Dashboard, reason: None